In [8]:
import pandas as pd
import numpy as np

# Load BMI data
BMI = pd.read_csv('bmi.csv')

# Preprocess data (assuming binary classification: Normal or not)
BMI['Target'] = BMI['Index'].apply(lambda x: 1 if x == 3 else 0)

# Define Genetic Algorithm parameters
population_size = 50
num_generations = 50
mutation_rate = 0.1

# Initialize population randomly
population = np.random.randint(2, size=(population_size, len(BMI.columns) - 1))

# Define fitness function
def fitness(individual):
    # Assume a simple fitness function based on correct predictions
    predictions = np.sum(individual == BMI['Target'].values[:, np.newaxis], axis=1)
    return np.sum(predictions == 1)

# Genetic Algorithm
for generation in range(num_generations):
    # Evaluate fitness of each individual
    fitness_values = np.apply_along_axis(fitness, 1, population)

    # Select parents based on fitness
    parents_indices = np.argsort(fitness_values)[-10:]  # Select top 10 individuals as parents
    parents = population[parents_indices]

    # Crossover
    crossover_point = len(BMI.columns) // 2
    children = []
    for _ in range(population_size - len(parents)):
        parent1, parent2 = np.random.choice(parents_indices, 2, replace=False)
        child = np.concatenate((population[parent1][:crossover_point], population[parent2][crossover_point:]))
        children.append(child)

    # Mutation
    mutation_mask = np.random.rand(population_size, len(BMI.columns) - 1) < mutation_rate
    population = np.logical_xor(population, mutation_mask).astype(int)

    # Replace old population with new population
    population[:len(parents)] = parents
    population[len(parents):] = children

# Select the best individual from the final generation
best_individual = population[np.argmax(np.apply_along_axis(fitness, 1, population))]

# Use the best individual for prediction
new_data = np.array([[170, 80, 1]])  # Example new data
prediction = np.sum(best_individual[:-1] == (new_data > 0))
predicted_bmi = "Normal" if prediction else "Not Normal"

print("Predicted BMI:", predicted_bmi)

Predicted BMI: Normal
